In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("rope-mpro.csv")
df.head()

,Unnamed: 0,t1:CA-N-C-CA,t1:CD2-CG-CB-CA,t1:CE1-ND1-CG-CB,t1:CE1-ND1-CG-CD2,t1:CG-CB-CA-C,t1:CG-CD2-NE2-CE1,t1:N-C-CA-CB,t1:N-CA-C-N,t1:N-CA-CB-CG,...,t4392:CD-CG-CB-CA,t4392:CG-CB-CA-C,t4392:N-CA-CB-CG,t4392:NE2-CD-CG-CB,t4392:O-C-CA-CB,t4392:O-C-CA-N,t4392:OE1-CD-CG-CB,t4392:OXT-C-CA-CB,t4392:OXT-C-CA-N,Unnamed: 4810
0,theseus_z7vh8_0A_chainA_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,theseus_z7tll_0B_chainB_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,theseus_z7tll_0B_chainA_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,theseus_z7tll_0A_chainB_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,theseus_z7tll_0A_chainA_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# filter out only the phi, psi, and omega angles
names = np.array(
    ["Unnamed: 0"]
    + [k for k in df.columns if "C-N-CA-C" in k or "N-CA-C-N" in k or "CA-N-C-CA" in k]
)
bb1 = df.copy()
bb1 = bb1[names]

# select the columns and show
bb1.rename(columns={"Unnamed: 0": "Name"}, inplace=True)
bb1.shape

(2845, 1095)

In [4]:
bb1.head()

,Name,t1:CA-N-C-CA,t1:N-CA-C-N,t2:C-N-CA-C,t2:CA-N-C-CA,t2:N-CA-C-N,t919:CA-N-C-CA,t919:N-CA-C-N,t920:C-N-CA-C,t920:CA-N-C-CA,...,t4389:C-N-CA-C,t4389:CA-N-C-CA,t4389:N-CA-C-N,t4390:C-N-CA-C,t4390:CA-N-C-CA,t4390:N-CA-C-N,t4391:C-N-CA-C,t4391:CA-N-C-CA,t4391:N-CA-C-N,t4392:C-N-CA-C
0,theseus_z7vh8_0A_chainA_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,theseus_z7tll_0B_chainB_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,theseus_z7tll_0B_chainA_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,theseus_z7tll_0A_chainB_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,theseus_z7tll_0A_chainA_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# select only within the resnum_bounds
resnum_bounds = (3265, 3562)
print(resnum_bounds[1] - resnum_bounds[0])
get_columns = np.array(
    [
        x
        for x in bb1.columns[1:]
        if int(x.split(":")[0][1:]) >= resnum_bounds[0]
        and int(x.split(":")[0][1:]) <= resnum_bounds[1]
    ]
)

# filtering step
filtering = list()

# check if the boundaries have any NaNs
for i, col in enumerate(get_columns):
    # check the boundary conditions
    if (
        int(col.split(":")[0][1:]) == resnum_bounds[0]
        or int(col.split(":")[0][1:]) == resnum_bounds[1]
    ):
        # if there are NaNs return False
        if np.any(bb1[col].isna()):
            filtering.append(False)

        else:
            filtering.append(True)

    # otherwise always keep the column for reconstruction
    else:
        filtering.append(True)

# filter the DataFrame according to the desired filtering from above
filtering = np.array(filtering)
bb1 = bb1[np.hstack([bb1.columns[0], get_columns[np.array(filtering)]])]
print(bb1.shape)
bb1.head()

297
(2845, 889)


,Name,t3266:C-N-CA-C,t3266:CA-N-C-CA,t3266:N-CA-C-N,t3267:C-N-CA-C,t3267:CA-N-C-CA,t3267:N-CA-C-N,t3268:C-N-CA-C,t3268:CA-N-C-CA,t3268:N-CA-C-N,...,t3558:N-CA-C-N,t3559:C-N-CA-C,t3559:CA-N-C-CA,t3559:N-CA-C-N,t3560:C-N-CA-C,t3560:CA-N-C-CA,t3560:N-CA-C-N,t3561:C-N-CA-C,t3561:CA-N-C-CA,t3561:N-CA-C-N
0,theseus_z7vh8_0A_chainA_A,-140.525,-176.391,119.312,-134.279,169.706,141.662,-51.4204,-164.970,110.507,...,-48.1937,-61.7547,177.561,-52.1385,-55.6581,-172.087,-39.9971,-73.5024,178.275,-34.7409
1,theseus_z7tll_0B_chainB_B,-115.618,-177.178,127.669,-138.644,166.469,141.261,-59.7710,-156.768,127.946,...,-49.5144,-66.2686,177.266,-43.7373,-63.2884,-178.870,-48.8221,-57.4258,184.857,-42.2850
2,theseus_z7tll_0B_chainA_A,-135.730,-180.978,120.113,-133.256,167.007,141.685,-57.3101,-159.252,125.088,...,-40.3174,-69.1634,178.881,-48.9024,-58.8150,-175.994,-25.5221,-72.8429,184.825,-23.3427
3,theseus_z7tll_0A_chainB_B,-115.681,-177.138,127.666,-138.528,166.158,140.579,-58.6720,-157.397,127.167,...,-50.1784,-65.9917,177.218,-43.7445,-63.2210,-178.687,-48.9581,-57.5175,184.873,-41.8331
4,theseus_z7tll_0A_chainA_A,-135.740,-180.279,119.960,-133.396,168.209,142.341,-59.7792,-157.977,125.055,...,-40.3701,-69.1589,178.787,-48.8798,-58.7367,-176.127,-25.8095,-73.1505,184.631,-22.7562


In [6]:
# remove any chains that do not have complete dihedrals across the board
bb1 = bb1.dropna(axis=0, how="any")
bb1.shape

(2380, 889)

In [7]:
# save the new DataFrame
bb1.to_csv("filtered-mpro.csv")